<a href="https://colab.research.google.com/github/amura/deep-rl-robots/blob/main/Deep_RL_learning_robot%20-%20Module%203.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


#Rendering dependencies


In [1]:
!apt-get install -y xvfb python-opengl ffmpeg x11-utils > /dev/null 2>&1
!apt-get update > /dev/null 2>&1
!apt-get install cmake > /dev/null 2>&1
!pip install --upgrade setuptools > /dev/null 2>&1
!apt-get install swig > /dev/null 2>&1
!pip install gym pyvirtualdisplay > /dev/null 2>&1
!pip install ez_setup > /dev/null 2>&1
!pip install gym[atari] > /dev/null 2>&1
!pip install gym[box2d]==0.17.* \
            pyvirtualdisplay==0.2.* \
            PyOpenGL==3.1.* \
            PyOpenGL-accelerate==3.1.* > /dev/null 2>&1



In [2]:
#!pip install gym[all] > /dev/null 2>&1
!pip install Box2D /dev/null 2>&1

ERROR: Exception:
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/pip/_vendor/packaging/requirements.py", line 93, in __init__
    req = REQUIREMENT.parseString(requirement_string)
  File "/usr/local/lib/python3.7/dist-packages/pip/_vendor/pyparsing.py", line 1939, in parseString
    raise exc
  File "/usr/local/lib/python3.7/dist-packages/pip/_vendor/pyparsing.py", line 1929, in parseString
    loc, tokens = self._parse(instring, 0)
  File "/usr/local/lib/python3.7/dist-packages/pip/_vendor/pyparsing.py", line 1669, in _parseNoCache
    loc, tokens = self.parseImpl(instring, preloc, doActions)
  File "/usr/local/lib/python3.7/dist-packages/pip/_vendor/pyparsing.py", line 4037, in parseImpl
    loc, exprtokens = e._parse(instring, loc, doActions)
  File "/usr/local/lib/python3.7/dist-packages/pip/_vendor/pyparsing.py", line 1669, in _parseNoCache
    loc, tokens = self.parseImpl(instring, preloc, doActions)
  File "/usr/local/lib/python3.7/dist-package

In [3]:
from pyvirtualdisplay import Display
import logging
FORMAT = "%(asctime)s - %(name)s - %(levelname)s - %(message)s"
logging.basicConfig(level=logging.DEBUG, format=FORMAT)

!echo $DISPLAY

_display = Display(visible=0, size=(1400, 900))
_display.start()

!echo $DISPLAY


2021-05-05 06:26:57,092 - easyprocess - DEBUG - command: ['Xvfb', '-help']
2021-05-05 06:26:57,110 - easyprocess - DEBUG - process was started (pid=2896)
2021-05-05 06:26:57,195 - easyprocess - DEBUG - process has ended, return code=0
2021-05-05 06:26:57,199 - easyprocess - DEBUG - command: ['Xvfb', '-br', '-nolisten', 'tcp', '-screen', '0', '1400x900x24', ':1004']
2021-05-05 06:26:57,200 - easyprocess - DEBUG - command: ['Xvfb', '-br', '-nolisten', 'tcp', '-screen', '0', '1400x900x24', ':1005']
2021-05-05 06:26:57,210 - easyprocess - DEBUG - process was started (pid=2897)
2021-05-05 06:26:57,212 - pyvirtualdisplay.abstractdisplay - DEBUG - DISPLAY=:1005
2021-05-05 06:26:57,214 - easyprocess - DEBUG - command: ['xdpyinfo']
2021-05-05 06:26:57,235 - easyprocess - DEBUG - process was started (pid=2898)
2021-05-05 06:26:57,865 - easyprocess - DEBUG - process has ended, return code=0
2021-05-05 06:26:57,867 - pyvirtualdisplay.abstractdisplay - INFO - Successfully started X with display ":1

:1005


In [4]:
# Test run
import gym
from gym.envs.box2d import car_racing
from gym import logger as gymlogger
from gym import wrappers

gymlogger.setLevel(gymlogger.MIN_LEVEL)


env = gym.make('CarRacing-v0')
env = wrappers.Monitor(env,"./gym-results", force=True)
env.reset()
for _ in range(500):
    action = env.action_space.sample()
    #print(action)
    #env.render()    
    observation,reward, done, info = env.step(action)
    #print(observation)
    #print(reward)
    if done: break
env.close()
#print(env.action_space)
#print(env.observation_space)

/usr/local/lib/python3.7/dist-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


Track generation: 1196..1499 -> 303-tiles track


In [5]:
!echo $DISPLAY
#_display.stop()

#!echo $DISPLAY

:1005


In [6]:
def show_video(path):
    if not os.path.isfile(path):
        raise NameError("Cannot access{}".format(path))
    video = io.open(path, 'r+b').read()
    encoded = base64.b64encode(video)
    disp(HTML(data="""<video alt="test" controls><source src="data:video/mp4;base64,{0}" type="video/mp4"/></video> """.format(encoded.decode('ascii'))))


In [7]:
import io
import os
import base64
from IPython.display import HTML, display as disp

print(env.file_infix)
show_video('./gym-results/openaigym.video.%s.video000000.mp4' % env.file_infix)


0.1894


In [8]:
import math
import random
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from collections import namedtuple
from itertools import count
from PIL import Image

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision.transforms as T


random.seed(10) # For repeatability

# set up matplotlib
is_ipython = 'inline' in matplotlib.get_backend()
if is_ipython:
    from IPython import display

plt.ion()

# if gpu is to be used
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")



In [9]:
# Action space - discretise

# Turn_left	
# [ -1.0, 0.0, 0.0 ]
# Turn_right	
# [ +1.0, 0.0, 0.0 ]
# Brake	
# [ 0.0, 0.0, 0.8 ]
# Accelerate	
# [ 0.0, 1.0, 0.8 ]
# Do-Nothing	
# [ 0.0, 0.0, 0.0 ]

ACTIONS = [[ -1.0, 0.0, 0.0 ], [ +1.0, 0.0, 0.0 ], [ 0.0, 0.0, 0.8 ], [ 0.0, 0.0, 0.0 ]]
NUM_ACTIONS = len(ACTIONS)




In [10]:
resize = T.Compose([T.ToPILImage(),
                    T.Resize(40, interpolation=Image.CUBIC),
                    T.ToTensor()])

def get_screen():
    # Returned screen requested by gym is 400x600x3, but is sometimes larger
    # such as 800x1200x3. Transpose it into torch order (CHW).
    screen = env.render(mode='rgb_array').transpose((2, 0, 1))
    # Cart is in the lower half, so strip off the top and bottom of the screen
    _, screen_height, screen_width = screen.shape
    screen = screen[:, int(screen_height*0.4):int(screen_height * 0.8)]

    #view_width = int(screen_width * 0.6)      
    # Strip off the edges, so that we have a square image centered on a cart
    screen = screen[:, :, screen_width//3:2*screen_width//3]
    # Convert to float, rescale, convert to torch tensor
    # (this doesn't require a copy)
    screen = np.ascontiguousarray(screen, dtype=np.float32) / 255
    screen = torch.from_numpy(screen)
    # Resize, and add a batch dimension (BCHW)
    return resize(screen).unsqueeze(0).to(device)




/usr/local/lib/python3.7/dist-packages/torchvision/transforms/transforms.py:258: UserWarning: Argument interpolation should be of type InterpolationMode instead of int. Please, use InterpolationMode enum.
  "Argument interpolation should be of type InterpolationMode instead of int. "


Deep Q network


In [11]:
class DQN(nn.Module):

    def __init__(self, h, w, outputs):
        super(DQN, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, kernel_size=5, stride=2)
        self.bn1 = nn.BatchNorm2d(16)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=5, stride=2)
        self.bn2 = nn.BatchNorm2d(32)
        self.conv3 = nn.Conv2d(32, 32, kernel_size=5, stride=2)
        self.bn3 = nn.BatchNorm2d(32)

        # Number of Linear input connections depends on output of conv2d layers
        # and therefore the input image size, so compute it.
        def conv2d_size_out(size, kernel_size = 5, stride = 2):
            return (size - (kernel_size - 1) - 1) // stride  + 1
        convw = conv2d_size_out(conv2d_size_out(conv2d_size_out(w)))
        convh = conv2d_size_out(conv2d_size_out(conv2d_size_out(h)))
        linear_input_size = convw * convh * 32
        self.head = nn.Linear(linear_input_size, outputs)

    # Called with either one element to determine next action, or a batch
    # during optimization. Returns tensor([[left0exp,right0exp]...]).
    def forward(self, x):
        x = x.to(device)
        x = F.relu(self.bn1(self.conv1(x)))
        x = F.relu(self.bn2(self.conv2(x)))
        x = F.relu(self.bn3(self.conv3(x)))
        return self.head(x.view(x.size(0), -1))

In [12]:
#class Fully connected Q-function, state in values out
class FCQ(nn.Module):
    def __init__(self, 
                 input_dim, 
                 output_dim, 
                 hidden_dims=(32,32), 
                 activation_fc=F.relu):
        super(FCQ, self).__init__()
        self.activation_fc = activation_fc

        self.input_layer = nn.Linear(input_dim, hidden_dims[0])
        self.hidden_layers = nn.ModuleList()
        for i in range(len(hidden_dims)-1):
            hidden_layer = nn.Linear(hidden_dims[i], hidden_dims[i+1])
            self.hidden_layers.append(hidden_layer)
        self.output_layer = nn.Linear(hidden_dims[-1], output_dim)

        device = "cpu"
        if torch.cuda.is_available():
            device = "cuda:0"
        self.device = torch.device(device)
        self.to(self.device)
        
    def _format(self, state):
        x = state
        if not isinstance(x, torch.Tensor):
            x = torch.tensor(x, 
                             device=self.device, 
                             dtype=torch.float32)
            x = x.unsqueeze(0)
        return x

    def forward(self, state):
        x = self._format(state)
        x = self.activation_fc(self.input_layer(x))
        for hidden_layer in self.hidden_layers:
            x = self.activation_fc(hidden_layer(x))
        x = self.output_layer(x)
        return x
    
    def numpy_float_to_device(self, variable):
        variable = torch.from_numpy(variable).float().to(self.device)
        return variable
    
    def load(self, experiences):
        states, actions, new_states, rewards, is_terminals = experiences
        states = torch.from_numpy(states).float().to(self.device)
        actions = torch.from_numpy(actions).long().to(self.device)
        new_states = torch.from_numpy(new_states).float().to(self.device)
        rewards = torch.from_numpy(rewards).float().to(self.device)
        is_terminals = torch.from_numpy(is_terminals).float().to(self.device)
        return states, actions, new_states, rewards, is_terminals

In [13]:
# Hyper parameters
BATCH_SIZE = 128
GAMMA = 0.999
EPS_START = 0.9
EPS_END = 0.05
EPS_DECAY = 200
TARGET_UPDATE = 10

env.reset()

init_screen = get_screen()
_, _, screen_height, screen_width = init_screen.shape

# Get number of actions from gym action space
n_actions = NUM_ACTIONS
policy_net = DQN(screen_height, screen_width, n_actions).to(device)
optimizer = optim.RMSprop(policy_net.parameters())

steps_done = 0

# Epsilon-greedy for action selection
def select_action(state):
    global steps_done
    sample = random.random()
    eps_threshold = EPS_END + (EPS_START - EPS_END) * \
        math.exp(-1. * steps_done / EPS_DECAY)
    steps_done += 1
    if sample > eps_threshold:
        with torch.no_grad():
            # t.max(1) will return largest column value of each row.
            # second column on max result is index of where max element was
            # found, so we pick action with the larger expected reward.
            return policy_net(state).max(1)[1].view(1, 1)
    else:
        return torch.tensor([[random.randrange(n_actions)]], device=device, dtype=torch.long)

episode_durations = []

def plot_durations():
    plt.figure(2)
    plt.clf()
    durations_t = torch.tensor(episode_durations, dtype=torch.float)
    plt.title('Training...')
    plt.xlabel('Episode')
    plt.ylabel('Duration')
    plt.plot(durations_t.numpy())
    # Take 100 episode averages and plot them too
    if len(durations_t) >= 100:
        means = durations_t.unfold(0, 100, 1).mean(1).view(-1)
        means = torch.cat((torch.zeros(99), means))
        plt.plot(means.numpy())

    plt.pause(0.001)  # pause a bit so that plots are updated
    if is_ipython:
        display.clear_output(wait=True)
        display.display(plt.gcf())
        
episode_returns = []
def plot_returns():
    plt.pause(0.01)  # pause a bit so that plots are updated
    if is_ipython:
        display.clear_output(wait=True)
    plt.figure(2)
    plt.clf()
    returns = torch.tensor(episode_returns, dtype=torch.float)
    plt.title('Training...')
    plt.xlabel('Episode')
    plt.ylabel('Returns')
    plt.plot(returns.numpy(), label='Returns')
    if len(returns) >= 10:
        means = returns.unfold(0, 10, 1).mean(1).view(-1)
        means = torch.cat((torch.zeros(9), means))
        plt.plot(means.numpy(), label='10-episode-mean returns')
    plt.legend()

Track generation: 1384..1734 -> 350-tiles track


Following from solution

In [14]:
Transition = namedtuple('Transition',
                        ('state', 'action', 'next_state', 'reward'))


class BatchMemory(object):

    def __init__(self, capacity):
        self.capacity = capacity
        self.memory = []
        self.position = 0

    def push(self, *args):
        """Saves a transition."""
        if len(self.memory) < self.capacity:
            self.memory.append(None)
        self.memory[self.position] = Transition(*args)
        self.position = (self.position + 1) % self.capacity

    def sample(self, batch_size):
        return self.memory[:batch_size]

    def __len__(self):
        return len(self.memory)

memory = BatchMemory(500)

In [15]:
def optimize_model():
    if len(memory) < BATCH_SIZE:
        return
    transitions = memory.sample(BATCH_SIZE)
    # Transpose the batch (see https://stackoverflow.com/a/19343/3343043 for
    # detailed explanation). This converts batch-array of Transitions
    # to Transition of batch-arrays.
    batch = Transition(*zip(*transitions))

    # Compute a mask of non-final states and concatenate the batch elements
    # (a final state would've been the one after which simulation ended)
    non_final_mask = torch.tensor(tuple(map(lambda s: s is not None,
                                          batch.next_state)), device=device, dtype=torch.bool)
    non_final_next_states = torch.cat([s for s in batch.next_state
                                                if s is not None])
    state_batch = torch.cat(batch.state)
    action_batch = torch.cat(batch.action)
    reward_batch = torch.cat(batch.reward)

    # Compute Q(s_t, a) - the model computes Q(s_t), then we select the
    # columns of actions taken. These are the actions which would've been taken
    # for each batch state according to policy_net
    state_action_values = policy_net(state_batch).gather(1, action_batch)

    next_state_values = torch.zeros(BATCH_SIZE, device=device)
    next_state_values[non_final_mask] = policy_net(non_final_next_states).max(1)[0].detach()
    # Compute the expected Q values
    expected_state_action_values = (next_state_values * GAMMA) + reward_batch

    # Compute Huber loss
    loss = F.mse_loss(state_action_values, expected_state_action_values.unsqueeze(1))

    # Optimize the model
    optimizer.zero_grad()
    loss.backward()
    for param in policy_net.parameters():
        param.grad.data.clamp_(-1, 1)
    optimizer.step()

In [ ]:
num_episodes = 20

# wait until env zooms to car
def wait_for_zoom():
    for x in range(0, 50):
        env.step([0, 0, 0])

for i_episode in range(num_episodes):
    wait_for_zoom()
    rewards_in_episode = 0
    print('Episode: '+str(i_episode))
    # Initialize the environment and state
    env.reset()
    last_screen = get_screen()
    _, reward, done, _ = env.step([0,1,0])
    current_screen = get_screen()
    state = current_screen - last_screen
    for t in count():
        # Select and perform an action
        action = select_action(state)
        _, reward, done, _ = env.step(ACTIONS[action.item()])
        rewards_in_episode = rewards_in_episode + reward
        reward = torch.tensor([reward], device=device)

        # Observe new state
        last_screen = current_screen
        current_screen = get_screen()
        if not done:
            next_state = current_screen - last_screen
        else:
            next_state = None

        # Store the transition in memory
        memory.push(state, action, next_state, reward)

        # Move to the next state
        state = next_state

        # Perform one step of the optimization
        optimize_model()
        if done:
            episode_returns.append(rewards_in_episode)
            print('Episode returns: ' + str(rewards_in_episode))
            torch.save(policy_net.state_dict(), './policy_net')
            if i_episode == (num_episodes - 1):
              plot_returns()
            break

print('Complete')
env.render()
env.close()
plt.ioff()
plt.show()

Episode: 0
Track generation: 1113..1400 -> 287-tiles track
Episode returns: -99.89999999999858
Episode: 1
Track generation: 1258..1577 -> 319-tiles track
Episode returns: -99.89999999999858
Episode: 2
Track generation: 1136..1424 -> 288-tiles track
Episode returns: -96.4156794425075
Episode: 3
Track generation: 1085..1366 -> 281-tiles track
Episode returns: -99.89999999999858
Episode: 4
Track generation: 1189..1497 -> 308-tiles track
Episode returns: -99.89999999999858
Episode: 5
Track generation: 1196..1499 -> 303-tiles track
Episode returns: -99.89999999999858
Episode: 6
Track generation: 1232..1544 -> 312-tiles track
Episode returns: -99.89999999999858
Episode: 7
Track generation: 1315..1647 -> 332-tiles track
Episode returns: -99.89999999999858
Episode: 8
Track generation: 1304..1634 -> 330-tiles track


In [ ]:
loaded_policy_net = DQN(screen_height, screen_width, n_actions).to(DEVICE)
loaded_policy_net.load_state_dict(torch.load('./policy_net'))
loaded_policy_net.eval()

In [ ]:
from gym import wrappers

done = False
env = wrappers.Monitor(env, "./gym-results", force=True)
env.reset()
wait_for_zoom()
last_screen = get_screen()

# Push the car forward so that current screen and last screen are not equal
env.step([0, 1, 0])

current_screen = get_screen()
state = current_screen - last_screen

while not done:
    with torch.no_grad():

    _, reward, done, _ = env.step(ACTIONS[loaded_policy_net(state).max(1)[1].view(1, 1).item()])

    reward = torch.tensor([reward], device=device)

    # Observe new state
    last_screen = current_screen
    current_screen = get_screen()
    if not done:
        next_state = current_screen - last_screen
    else:
        next_state = None

    # Move to the next state
    state = next_state
    if done: break

env.close()

In [ ]:
print(env.file_infix)
show_video('./gym-results/openaigym.video.%s.video000000.mp4' % env.file_infix)